In [186]:
import numpy as np
import pandas as pd
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [187]:
'''def vector_method1(model, vector_features, new_data):
    
    index = nmslib.init(space='cosinesimil', method='sw-graph')
    nmslib.addDataPointBatch(index, np.arange(model.wv.syn0.shape[0], dtype=np.int32), model.wv.syn0)
    index.createIndex({}, print_progress=True)
    start = time.time()
    items = nmslib.knnQuery(index, 10, king_vec.tolist())
    print(time.time() - start)
    print([model.wv.index2word[idx] for idx in items])'''

"def vector_method1(model, vector_features, new_data):\n    \n    index = nmslib.init(space='cosinesimil', method='sw-graph')\n    nmslib.addDataPointBatch(index, np.arange(model.wv.syn0.shape[0], dtype=np.int32), model.wv.syn0)\n    index.createIndex({}, print_progress=True)\n    start = time.time()\n    items = nmslib.knnQuery(index, 10, king_vec.tolist())\n    print(time.time() - start)\n    print([model.wv.index2word[idx] for idx in items])"

In [188]:
def vector_method3(model, vector_features, X_test, N_QUERY_RESULT=10):
    
    '''new_data - X_test without forward. Data goes through model and becomes a new vector
       which we compare with elements in vector_features'''
    
    new_data = model(X_test)
    new_data_np = np.array(new_data)
    
    vector_features_np = np.array([np.array(vector) for vector in vector_features])
    
    if vector_features_np.ndim == 1:
        vector_features_np = vector_features[:, np.newaxis]
      
    nbrs = NearestNeighbors(n_neighbors=N_QUERY_RESULT, metric="cosine").fit(vector_features)
    
    kneighbors = []
    for data in new_data_np:
        
        data = data[np.newaxis, :]
        
        distances, indices = nbrs.kneighbors(data)
        similar_image_indices = indices.reshape(-1)
        kneighbors.append(vector_features[similar_image_indices])
    
    return kneighbors

In [189]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(15, 12))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
    
def visual_kneighbors(kneighbors):
    # Расположим картинки рядом
    for part in kneighbors:
        out = torchvision.utils.make_grid(kneighbors)
        imshow(out)     